In [31]:
import pandas as pd
from selenium import webdriver
from selenium.webdriver.chrome.service import Service as ChromeService
from selenium.webdriver.support.ui import WebDriverWait
from webdriver_manager.chrome import ChromeDriverManager
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.common.by import By
from bs4 import BeautifulSoup
import requests
import re
import numpy as np

job_titles = []
company_names = []
company_ratings = []
company_locations = []
job_pays = []
descriptions = []
posted = []
current_page = 1
url = 'https://www.indeed.com/jobs?q=Data+Science&l=Los+Angeles%2C+CA&vjk=74cadd03cb0dd918'
driver = webdriver.Chrome(service=ChromeService(ChromeDriverManager().install()))
driver.get(url)
pagination = driver.find_element(By.XPATH, ".//nav[contains(@role, 'navigation')]")
last_page = pagination.find_elements(By.XPATH, ".//a[contains(@data-testid, 'pagination-page-5')]")[-1].text
driver.set_page_load_timeout(10)

for page_num in range(1, int(last_page) + 11):
    container = driver.find_element(By.XPATH, ".//div[contains(@id, 'mosaic-jobResults')]")
    all_jobs = container.find_elements(By.XPATH, ".//div[contains(@class, 'job_seen_beacon')]")
    for job in all_jobs:
        title = job.find_element(By.XPATH, ".//h2[contains(@class, 'jobTitle')]").text
        try:
            location = job.find_element(By.XPATH, ".//div[contains(@class, 'company_location')]").text.split("\n")[1]
        except:
            location = job.find_element(By.XPATH, ".//div[contains(@class, 'company_location')]").text.split("\n")[0]
        company = job.find_element(By.XPATH, ".//div[contains(@class, 'company_location')]").text.split("\n")[0]
        when_posted= job.find_element(By.XPATH, ".//tr[contains(@class, 'underShelfFooter')]").text.split("\n")[-1].strip("More...")
        description = job.find_element(By.XPATH, ".//tr[contains(@class, 'underShelfFooter')]").text.split("\n")[0]
        try:
            string = job.find_element(By.XPATH, ".//div[contains(@class, 'salary-snippet-container')]").text
            listy = string.split()
            for element in listy:
                if "$" in element:
                    pay = element
        except:
            pay = "NaN"
        job_titles.append(title)
        job_pays.append(pay)
        company_names.append(company)
        descriptions.append(description)
        company_locations.append(location)
        posted.append(when_posted)
    page_num += 1

    try:
        #next = driver.find_element(By.XPATH, ".//a[contains(@aria-label, 'Next Page')]")
        next = WebDriverWait(driver, 10).until(EC.element_to_be_clickable((By.XPATH, ".//a[contains(@aria-label, 'Next Page')]")))
        print(next)
        next.click()
    except:
        #next = False
        print(f'failed {page_num}')
        pass

<selenium.webdriver.remote.webelement.WebElement (session="b508649f3b3bffe1ca7dc8eb8f7f0210", element="f.DA3254FA6EF9C8106629176EF1B0A021.d.FEC8B9A48FCE95BACDD062CEC274632C.e.1565")>
<selenium.webdriver.remote.webelement.WebElement (session="b508649f3b3bffe1ca7dc8eb8f7f0210", element="f.DA3254FA6EF9C8106629176EF1B0A021.d.06EA097517096D52A49A3B7288D91DB8.e.5341")>
<selenium.webdriver.remote.webelement.WebElement (session="b508649f3b3bffe1ca7dc8eb8f7f0210", element="f.DA3254FA6EF9C8106629176EF1B0A021.d.0A3FFE37F2E3C1C3C59B8729274407E2.e.9519")>
<selenium.webdriver.remote.webelement.WebElement (session="b508649f3b3bffe1ca7dc8eb8f7f0210", element="f.DA3254FA6EF9C8106629176EF1B0A021.d.A7CC32122130867BFBD1264654FF0F85.e.13903")>
<selenium.webdriver.remote.webelement.WebElement (session="b508649f3b3bffe1ca7dc8eb8f7f0210", element="f.DA3254FA6EF9C8106629176EF1B0A021.d.C0B5A6C11FD4E29A63B19AE20E0FBAC5.e.18181")>
<selenium.webdriver.remote.webelement.WebElement (session="b508649f3b3bffe1ca7dc8eb

In [32]:
#separate into city and state
cities = []
states = []
# Parse location data
for location in company_locations:
    parts = location.split(', ')
    if len(parts) > 1:
        city = parts[0]
        state = parts[1] if len(parts) > 1 else None
        cities.append(city)
        states.append(state[:2])
    else:
        cities.append(np.nan)
        states.append(np.nan)


#make salaries integers
salaries = []
for salary in job_pays:
    if salary == "NaN" or salary == "" :
        salaries.append(np.nan)
    else:
        salary = salary.replace('$', '')
        salary = salary.replace(',', '')
        salary_list = salary.split('.')
        if len(salary_list[0]) <= 3:
            #Then I know it is hourly rate
            salaries.append(float(salary) * 40 * 52)
        else:
            salaries.append(float(salary))


job_info = pd.DataFrame({"Title": job_titles,
              "Company": company_names,
              "Location": company_locations,
              "Job Description": descriptions,
              "When Posted": posted,
              "Pay": salaries,
              "City":cities,
               "State": states})
#try getting hybrid or remote status
def is_hybrid_remote(location):
    return 'Hybrid' in location or 'Remote' in location

# Apply the function to create a new column 'Is_Hybrid_Remote'
job_info['Is_Hybrid_Remote'] = job_info['Location'].apply(is_hybrid_remote)

In [33]:
job_info

,Title,Company,Location,Job Description,When Posted,Pay,City,State,Is_Hybrid_Remote
0,Software Developer - AI Trainer (Contract),DataAnnotation,4.5,You will work with the chatbots that we are bu...,Active 8 days ag,83200.0,NaN,NaN,False
1,AI Training for Physics,Outlier,3.7,"Help train AI models to become more accurate, ...",Today,104000.0,NaN,NaN,False
2,Data Engineer,LegalMation,"Hybrid remote in Los Angeles, CA 90071","Experience with cloud technologies, NLP, and m...",Active 4 days ag,80000.0,Hybrid remote in Los Angeles,CA,True
3,Pharmaceutical Sales Representative,Inizio Engage,3.5,"Our passionate, global workforce augments loca...",Active 2 days ag,130000.0,NaN,NaN,False
4,Associate Data Analyst - Hybrid Remote,CEDARS-SINAI,"Hybrid remote in Los Angeles, CA",Uses data quality tools to proactively identif...,Today·,93787.2,Hybrid remote in Los Angeles,CA,True
...,...,...,...,...,...,...,...,...,...
220,Principal AM Process Development Engineer,Divergent 3D,"Torrance, CA",7)Assure proper formulation and design of expe...,Posted 30+ days ago·,210000.0,Torrance,CA,False
221,Master of Applied Statistics and Data Science ...,UCLA,"Los Angeles, CA",Ideal candidates will have a Ph.D. in Statisti...,Posted 21 days ago·,109516.0,Los Angeles,CA,False
222,"Scientific Data Analyst, Sr. / Heart Institute...",Children's Hospital Los Angeles,"Los Angeles, CA",To establish and support a data server and fac...,Posted 30+ days ago·,NaN,Los Angeles,CA,False
223,Data/GIS Analyst - Advanced Analytics Specialist,Hazen and Sawyer,"Los Angeles, CA 90017","Proficiency in database development, data anal...",Posted 30+ days ago·,85000.0,Los Angeles,CA,False


In [34]:
#job_info.to_csv("masterpiece_fixed_pay.csv")